# 机器学习预测模型

## 📋 概述

机器学习预测模型是股票分析中应用广泛的方法，通过使用各种机器学习算法来预测股票价格和收益。本章节将详细介绍机器学习预测模型在股票分析中的应用，包括随机森林、XGBoost、支持向量机等算法。

**学习方式**：本文件是Jupyter Notebook格式，你可以边看边运行代码，通过实际代码示例理解机器学习预测模型。


In [ ]:
# 安装必要的库（如果还没有安装）
# !pip install pandas numpy matplotlib scikit-learn xgboost

# 导入库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

# 尝试导入XGBoost
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False
    print("XGBoost不可用，将使用其他模型")

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境准备完成！")


## 📖 核心内容

### 1. 机器学习预测模型概述

#### 1.1 常用算法

**树模型**：
- 随机森林（Random Forest）
- 梯度提升（Gradient Boosting）
- XGBoost、LightGBM

**线性模型**：
- 线性回归
- 岭回归
- Lasso回归

**支持向量机**：
- SVR（支持向量回归）

让我们准备数据并训练多个模型：


In [ ]:
# 生成示例数据
np.random.seed(42)
n_days = 500
dates = pd.date_range(start='2022-01-01', periods=n_days, freq='D')
dates = dates[dates.weekday < 5]

prices = 10 + np.cumsum(np.random.randn(len(dates)) * 0.1)
volumes = np.random.randint(1000000, 10000000, len(dates))

price_data = pd.DataFrame({
    'Close': prices,
    'Volume': volumes
}, index=dates)

# 计算特征
price_data['Returns'] = price_data['Close'].pct_change()
price_data['MA5'] = price_data['Close'].rolling(window=5).mean()
price_data['MA20'] = price_data['Close'].rolling(window=20).mean()
price_data['Volatility'] = price_data['Returns'].rolling(window=20).std()
price_data['Momentum'] = price_data['Close'] / price_data['Close'].shift(5) - 1
price_data['Volume_Ratio'] = price_data['Volume'] / price_data['Volume'].rolling(window=20).mean()

# 目标变量：未来1期收益率
price_data['Future_Return'] = price_data['Returns'].shift(-1)

price_data = price_data.dropna()

# 准备特征和目标
feature_cols = ['Returns', 'MA5', 'MA20', 'Volatility', 'Momentum', 'Volume_Ratio']
X = price_data[feature_cols].values
y = price_data['Future_Return'].values

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=False)

# 标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"数据准备完成！")
print(f"训练集：{len(X_train)} 个样本")
print(f"测试集：{len(X_test)} 个样本")


In [ ]:
# 训练多个模型
models = {}

# 1. 随机森林
rf_model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
rf_model.fit(X_train_scaled, y_train)
models['随机森林'] = rf_model

# 2. 梯度提升
gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
gb_model.fit(X_train_scaled, y_train)
models['梯度提升'] = gb_model

# 3. XGBoost
if XGBOOST_AVAILABLE:
    xgb_model = xgb.XGBRegressor(n_estimators=100, max_depth=3, random_state=42)
    xgb_model.fit(X_train_scaled, y_train)
    models['XGBoost'] = xgb_model

# 4. 支持向量回归
svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
svr_model.fit(X_train_scaled, y_train)
models['SVR'] = svr_model

# 预测和评估
results = {}
for name, model in models.items():
    train_pred = model.predict(X_train_scaled)
    test_pred = model.predict(X_test_scaled)
    
    train_mse = mean_squared_error(y_train, train_pred)
    test_mse = mean_squared_error(y_test, test_pred)
    test_mae = mean_absolute_error(y_test, test_pred)
    test_r2 = r2_score(y_test, test_pred)
    
    results[name] = {
        'train_mse': train_mse,
        'test_mse': test_mse,
        'test_mae': test_mae,
        'test_r2': test_r2,
        'predictions': test_pred
    }
    
    print(f"\n{name}模型：")
    print(f"  训练集MSE: {train_mse:.6f}")
    print(f"  测试集MSE: {test_mse:.6f}")
    print(f"  测试集MAE: {test_mae:.6f}")
    print(f"  测试集R²: {test_r2:.4f}")

# 可视化预测结果
fig, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

test_dates = price_data.index[-len(y_test):]

# 实际值
axes[0].plot(test_dates, y_test * 100, 'o-', label='实际收益率', 
            linewidth=2, markersize=4, color='black', alpha=0.7)

# 各模型预测
colors = {'随机森林': 'green', '梯度提升': 'blue', 'XGBoost': 'orange', 'SVR': 'red'}
for name, result in results.items():
    if name in colors:
        axes[0].plot(test_dates, result['predictions'] * 100, 
                    label=f'{name}预测', linewidth=1.5, color=colors[name], alpha=0.7)

axes[0].axhline(y=0, color='gray', linestyle='--', linewidth=0.5)
axes[0].set_title('机器学习模型预测结果对比', fontsize=14, fontweight='bold')
axes[0].set_ylabel('收益率 (%)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 模型性能对比
model_names = list(results.keys())
mse_values = [results[name]['test_mse'] for name in model_names]
mae_values = [results[name]['test_mae'] for name in model_names]

x = np.arange(len(model_names))
width = 0.35

axes[1].bar(x - width/2, mse_values, width, label='MSE', alpha=0.7, color='skyblue')
axes[1].bar(x + width/2, mae_values, width, label='MAE', alpha=0.7, color='lightgreen')
axes[1].set_title('模型性能对比', fontsize=14, fontweight='bold')
axes[1].set_xlabel('模型')
axes[1].set_ylabel('误差')
axes[1].set_xticks(x)
axes[1].set_xticklabels(model_names, rotation=45, ha='right')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### 3. 特征重要性分析

让我们分析哪些特征最重要：


In [ ]:
# 特征重要性分析
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 随机森林特征重要性
if '随机森林' in models:
    rf_importance = models['随机森林'].feature_importances_
    indices = np.argsort(rf_importance)[::-1]
    
    axes[0].barh(range(len(feature_cols)), rf_importance[indices], 
                color='skyblue', alpha=0.7, edgecolor='black')
    axes[0].set_yticks(range(len(feature_cols)))
    axes[0].set_yticklabels([feature_cols[i] for i in indices])
    axes[0].set_xlabel('重要性')
    axes[0].set_title('随机森林特征重要性', fontsize=12, fontweight='bold')
    axes[0].grid(True, alpha=0.3, axis='x')

# 梯度提升特征重要性
if '梯度提升' in models:
    gb_importance = models['梯度提升'].feature_importances_
    indices = np.argsort(gb_importance)[::-1]
    
    axes[1].barh(range(len(feature_cols)), gb_importance[indices], 
                color='lightgreen', alpha=0.7, edgecolor='black')
    axes[1].set_yticks(range(len(feature_cols)))
    axes[1].set_yticklabels([feature_cols[i] for i in indices])
    axes[1].set_xlabel('重要性')
    axes[1].set_title('梯度提升特征重要性', fontsize=12, fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n特征重要性排名（随机森林）：")
if '随机森林' in models:
    rf_importance = models['随机森林'].feature_importances_
    indices = np.argsort(rf_importance)[::-1]
    for i, idx in enumerate(indices, 1):
        print(f"{i}. {feature_cols[idx]}: {rf_importance[idx]:.4f}")
